# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///df_clean.db')
df = pd.read_sql_table('df_clean', engine)
X = df['message']
Y = df.iloc[:, 4:]

In [9]:
Y.sum().sort_values(ascending  = False)[:5]

related            20282
aid_related        10860
weather_related     7297
direct_report       5075
request             4474
dtype: int64

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # normalize case and remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    #tokenize the text
    tokens = word_tokenize(text)
    # remove stop words and lemmatize the text
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
        ('multi_clf', MultiOutputClassifier(DecisionTreeClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [10]:
y_predict = pipeline.predict(X_test)

In [11]:
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test.values[:,i], y_predict[:, i]))

             precision    recall  f1-score   support

          0       0.47      0.45      0.46      1208
          1       0.83      0.85      0.84      3990
          2       0.43      0.52      0.47        46

avg / total       0.75      0.75      0.75      5244

             precision    recall  f1-score   support

          0       0.91      0.92      0.92      4356
          1       0.59      0.57      0.58       888

avg / total       0.86      0.86      0.86      5244

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5216
          1       0.06      0.04      0.04        28

avg / total       0.99      0.99      0.99      5244

             precision    recall  f1-score   support

          0       0.73      0.74      0.73      2996
          1       0.65      0.64      0.64      2248

avg / total       0.69      0.70      0.70      5244

             precision    recall  f1-score   support

          0       0.94      0.95 

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {'tfidf__use_idf':(True, False), 'multi_clf__estimator__max_depth': [5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, verbose = 5)

In [13]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=True ..........
[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=True, score=0.18395079387784294, total=   9.2s
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=True ..........


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    9.7s remaining:    0.0s


[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=True, score=0.19053068230582176, total=   9.1s
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=True ..........


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   19.2s remaining:    0.0s


[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=True, score=0.19127324749642347, total=   9.5s
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=False .........


...

[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=False, score=0.1815190959805464, total=   8.1s
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=False .........


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   37.8s remaining:    0.0s


[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=False, score=0.20097267915891862, total=   7.9s
[CV] multi_clf__estimator__max_depth=5, tfidf__use_idf=False .........
[CV]  multi_clf__estimator__max_depth=5, tfidf__use_idf=False, score=0.20758226037195995, total=   8.1s
[CV] multi_clf__estimator__max_depth=10, tfidf__use_idf=True .........
[CV]  multi_clf__estimator__max_depth=10, tfidf__use_idf=True, score=0.2065512802174224, total=  18.7s
[CV] multi_clf__estimator__max_depth=10, tfidf__use_idf=True .........
[CV]  multi_clf__estimator__max_depth=10, tfidf__use_idf=True, score=0.2118437991703619, total=  18.3s
[CV] multi_clf__estimator__max_depth=10, tfidf__use_idf=True .........
[CV]  multi_clf__estimator__max_depth=10, tfidf__use_idf=True, score=0.2034334763948498, total=  18.8s
[CV] multi_clf__estimator__max_depth=10, tfidf__use_idf=False ........
[CV]  multi_clf__estimator__max_depth=10, tfidf__use_idf=False, score=0.20626519811185812, total=  16.4s
[CV] multi_clf__estimat

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'multi_clf__estimator__max_depth': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_predict = pipeline.predict(X_test)
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test.values[:,i], y_predict[:, i]))

             precision    recall  f1-score   support

          0       0.47      0.45      0.46      1208
          1       0.83      0.85      0.84      3990
          2       0.43      0.52      0.47        46

avg / total       0.75      0.75      0.75      5244

             precision    recall  f1-score   support

          0       0.91      0.92      0.92      4356
          1       0.59      0.57      0.58       888

avg / total       0.86      0.86      0.86      5244

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5216
          1       0.06      0.04      0.04        28

avg / total       0.99      0.99      0.99      5244

             precision    recall  f1-score   support

          0       0.73      0.74      0.73      2996
          1       0.65      0.64      0.64      2248

avg / total       0.69      0.70      0.70      5244

             precision    recall  f1-score   support

          0       0.94      0.95 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('multi_clf', MultiOutputClassifier(RandomForestClassifier(class_weight = 'balanced')))])

In [16]:
parameters = {'vect__ngram_range': ((1,1), (1, 2)), 'multi_clf__estimator__min_samples_split':[2, 4]}
cv = GridSearchCV(pipeline, parameters, n_jobs = -1, verbose = 5)

In [17]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 
[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.23444428550994134, total= 2.2min
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s


[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.23444428550994134, total= 2.2min
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.5min remaining:    0.0s


[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.2317596566523605, total= 2.2min
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) 


...

[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.23644686024889144, total= 3.4min
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 14.2min remaining:    0.0s


[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.24474324131025604, total= 3.3min
[CV] multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) 
[CV]  multi_clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.23533619456366237, total= 3.3min
[CV] multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) 
[CV]  multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.23415820340437707, total= 2.1min
[CV] multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) 
[CV]  multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.23215562866542697, total= 2.0min
[CV] multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) 
[CV]  multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.2390557939914163, total= 2.1min
[CV] multi_clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2) 
[CV]  multi_clf__estimator__min_samples_split=4, ve

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 44.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'multi_clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

### 9. Export your model as a pickle file

In [18]:
with open('clf.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
s

In [ ]:
dfasdfasdf